# Heart Disease Exploration

Authors: Sophie Groenwold, Priyanka Banerjee

In [3]:
import numpy as np
import altair as alt
import pandas as pd

## Trying to use 1-4 target values

Scratch work.

In [4]:
kaggle_df = pd.read_csv('heart.csv')
# kaggle_df.head(10)
kaggle_df = kaggle_df.sort_values(by=['age'])
kaggle_df = kaggle_df.reset_index().drop('index', axis=1)
# print(kaggle_df['trestbps'].shape)
# print(kaggle_df.shape)

kaggle_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
1,34,1,3,118,182,0,0,174,0,0.0,2,0,2,1
2,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
3,35,1,0,126,282,0,0,156,1,0.0,2,0,3,0
4,35,0,0,138,183,0,1,182,0,1.4,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1
299,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1
300,74,0,1,120,269,0,0,121,1,0.2,2,1,2,1
301,76,0,2,140,197,0,2,116,0,1.1,1,0,2,1


In [ ]:
cleveland_df = pd.read_csv('processed_cleveland.csv')
# cleveland_df.head(10)
cleveland_df = cleveland_df.sort_values(by=['a'])
cleveland_df

In [ ]:
# to view all the range of values for each feature

# for col in kaggle_df:
#     print(kaggle_df[col].unique())

# for col in cleveland_df:
#     print(cleveland_df[col].unique())

In [ ]:
# age = a
# sex = b
# trestbps = d (but weird -- cleveland has an extra row)
# chol = e
# fbs = f
# thalach = h
# oldpeak = j

# kaggle values missing their counterpart:
# - cp
# - restecg
# - exang
# - slope
# - ca
# - thal
# - target (which is what we would've replaced)

# cleveland values missing their counterpart:
# - c
# - g
# - i
# - k
# - l
# - m
# - n (which is what we would've used in place of target)

In [ ]:
# I used this to compare the different columns from each dataframe

# kaggle_df = kaggle_df.sort_values(by=['ca'])
# cleveland_df = cleveland_df.sort_values(by=['l'])
# print(kaggle_df['restecg'].unique().shape)
# print(cleveland_df['j'].unique().shape)
# print(kaggle_df['ca'].unique() == cleveland_df['l'].unique())

## Exploration


In [9]:
kaggle_df[kaggle_df['sex'] == 0].shape #96 women
kaggle_df[kaggle_df['sex'] != 0].shape #207 men

(207, 14)

---

## PCA

Following similar procedures as from lab06.

In [145]:
# normalizing the data
kaggle_val = kaggle_df.drop('target', axis = 1).values
n = kaggle_val.shape[0]

kaggle_mean = np.mean(kaggle_val, axis = 0)
kaggle_mean
kaggle_sd = np.std(kaggle_val, axis = 0)
kaggle_sd
kaggle_centered = (kaggle_val - kaggle_mean) / (kaggle_sd)
kaggle_centered

array([[-2.7976235 ,  0.68100522,  0.03203122, ...,  0.97635214,
        -0.71442887, -0.51292188],
       [-2.24617937,  0.68100522,  1.97312292, ...,  0.97635214,
        -0.71442887, -0.51292188],
       [-2.24617937, -1.46841752,  0.03203122, ...,  0.97635214,
        -0.71442887, -0.51292188],
       ...,
       [ 2.16537369, -1.46841752,  0.03203122, ...,  0.97635214,
         0.26508221, -0.51292188],
       [ 2.38595135, -1.46841752,  1.00257707, ..., -0.64911323,
        -0.71442887, -0.51292188],
       [ 2.49624017,  0.68100522, -0.93851463, ...,  0.97635214,
         2.22410436, -0.51292188]])

In [134]:
# -0.05 < np.sum(kaggle_centered[0]) < 0.05 # make sure scaling was done correctly
# np.sum(kaggle_centered[0])
np.mean(kaggle_centered) # should equal 0
np.var(kaggle_centered) # should equal 1.0

0.9999999999999999

In [135]:
u, s, vt = np.linalg.svd(kaggle_centered, full_matrices=False)
u.shape, s, vt.shape

((303, 13),
 array([28.93435938, 21.57817595, 19.24886471, 18.91790396, 17.59704127,
        17.14394376, 16.16846585, 15.33334437, 14.75921501, 13.72354787,
        12.67366339, 11.32308003, 10.61479163]),
 (13, 13))

In [136]:
# find and compare total variance found from:

# (1) data
total_variance_data = sum(np.var(kaggle_df, axis=0))

# (2) singular values
total_variance_singular_values = sum(np.square(s)) / len(kaggle_df)

print(total_variance_data)
print(total_variance_singular_values)

3594.5453800825626
12.999999999999986


In [137]:
# variance explained by first singular value
variance_explained_by_1st_pc = (s[0]**2 / len(kaggle_centered)) / (total_variance_singular_values)
variance_explained_by_1st_pc

0.21254053121851954

In [138]:
PC_num = []
i = 1
n = len(s)
while i <= n:
    PC_num.append(i)
    i += 1

explained_var = pd.DataFrame({
    'PC #': PC_num, 
    'Fraction of Variance Explained' : (s**2 / len(kaggle_centered)) / (total_variance_singular_values)
})

print(explained_var)

# scree plot of variance explained by each PC
alt.Chart(explained_var, 
          title="Variance Explained by Principal Components"
).mark_bar(size=30).encode(
    alt.X('PC #:O'),
    alt.Y('Fraction of Variance Explained:Q')
).configure_axisX(labelAngle=0).properties(width=500)


    PC #  Fraction of Variance Explained
0      1                        0.212541
1      2                        0.118207
2      3                        0.094064
3      4                        0.090857
4      5                        0.078613
5      6                        0.074617
6      7                        0.066367
7      8                        0.059688
8      9                        0.055302
9     10                        0.047813
10    11                        0.040777
11    12                        0.032549
12    13                        0.028605


alt.Chart(...)

In [139]:
# relationships between PCs

P = pd.DataFrame(u @ np.diag(s))
P = P.rename(columns={0: "PC1", 1: "PC2", 2: "PC3", 3: "PC4", 4: "PC5", 5: "PC6", 6: "PC7",
                     7: "PC8", 8: "PC9", 9: "PC10", 10: "PC11", 11: "PC12", 12: "PC13", 13: "PC14"})
P['target'] = kaggle_df['target']

alt.Chart(P).mark_circle().encode(
    x = "PC1",
    y = "PC2",
    color = alt.Color('target:N', scale=alt.Scale(scheme='set1'))
)

alt.Chart(...)

In [140]:
# contributions of each feature kaggle_df on our PCs

def plot_pc(col_names, vt, k):
    """ 
    Plot how much each column of our data contributes 
    to each principal component and labels the rows of V^T.
    """
    df = pd.DataFrame({'v':vt[k, :], 'Column names':col_names})
    chart = alt.Chart(df).mark_bar().encode(
        x='Column names',
        y='v',
        opacity=alt.value(0.7)
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=14
    ).configure_axisX(
        labelAngle = 0
    ).properties(width = 700)
    return chart

# contribution of each feature for PC1
plot_pc(list(kaggle_df.drop('target', axis = 1).columns), vt, 0)

alt.Chart(...)

In [153]:
# contribution of each feature for PC2
plot_pc(list(kaggle_df.columns[:13]), vt, 1)

alt.Chart(...)

In [154]:
# contribution of each feature for PC3
plot_pc(list(kaggle_df.columns[:13]), vt, 2)

alt.Chart(...)

In [155]:
# contribution of each feature for PC4
plot_pc(list(kaggle_df.columns[:13]), vt, 3)

alt.Chart(...)

# Logistic Regression

In [ ]:
# from sklearn.linear_model import LogisticRegression
# logreg = LogisticRegression()

In [152]:
# X = pd.DataFrame(kaggle_centered)
# X.columns = kaggle_df.columns[:13]
# y = kaggle_df['target']
# y

0      1
1      1
2      1
3      0
4      1
      ..
298    1
299    1
300    1
301    1
302    0
Name: target, Length: 303, dtype: int64